In [2]:
from CS2_bo_functions import LHS_Design
from CS2_bo_functions import test_train_split
# from CS2_bo_functions import create_y_data
# from CS2_bo_functions import calc_y_exp
# from CS2_bo_functions import create_sse_data
from bo_functions_generic import create_y_data
from bo_functions_generic import calc_y_exp
from bo_functions_generic import create_sse_data
from bo_functions_generic import clean_1D_arrays, calc_muller
from scipy.stats import qmc
import torch
import numpy as np
import pandas as pd
import os

In [3]:
X = np.array([[1,1]])
print(X.reshape(-1,1))

x,y = X
print(x)

[[1]
 [1]]


ValueError: not enough values to unpack (expected 2, got 1)

In [4]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = False
seed = 9
LHS_Points =20
d = 8
# d = 2
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)

#Only going to regress "a and b" parameters
skip_param_types = 1 #(Skiiping A)
l_bounds =  [-2, -2, -10, -2, -2, -2,  5, -2]
u_bounds =  [ 2,  2,   0,  2,  2,  2, 15,  2]

# l_bounds =  [-2, -2]
# u_bounds =  [2, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)

In [6]:
#LHS for experimental data points
exp_LHS_Points = 10 #0 , 10, 15, 20, or 25 for 4, 8, 12, 16, and 24 params
exp_d = 2
exp_sampler = qmc.LatinHypercube(d=exp_d, seed = seed)
exp_sample = exp_sampler.random(n=exp_LHS_Points)

exp_l_bounds =  [-1.5, -0.5]
exp_u_bounds =  [1 , 2]
exp_LHS = qmc.scale(exp_sample, exp_l_bounds, exp_u_bounds)
print(exp_LHS)

[[-0.2175623   0.9282957 ]
 [-1.15078704 -0.44438352]
 [-1.42901866  0.27115497]
 [-0.96509841  1.77044059]
 [ 0.74335307  0.640688  ]
 [ 0.87876392  1.73371145]
 [-0.50140651  0.04234464]
 [ 0.00417444  1.05384754]
 [ 0.42109913 -0.17633343]
 [-0.32479527  1.31481288]]


In [7]:
#Creates Experimental data and saves to a csv
# Constants = np.array([-200,-100,-170,15,   #A
#                       -1,-1,-6.5,0.7,    #a
#                       0,0,11,0.6,        #b
#                       -10,-10,-6.5,0.7,  #c
#                       1,0,-0.5,-1,      #x0
#                       0,0.5,1.5,1])     #yo

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

# Theta_True = np.array([-2,2])
# Xexp = np.array([-2,-1,0,1,2])

Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

Xexp = np.vstack((exp_LHS, Xexp.T))
print(Xexp[0])
print(Xexp.shape)

n = Xexp.shape[0]
print(n)
noise_std = 0.01
Yexp = calc_y_exp(Constants, Xexp, noise_std)
print(Yexp)

[-0.2175623  0.9282957]
(15, 2)
15
[   5.05474681   74.53890398   12.24428821   16.62324988   44.33836208
  591.55992898   -4.72307241   26.04219455  -46.1040296   -52.96709878
 -146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]


In [5]:
df = pd.DataFrame({'X1exp': Xexp[:, 0], 'X2exp': Xexp[:, 1], 'Yexp':Yexp})
# df = pd.DataFrame({'Xexp': Xexp, 'Yexp':Yexp})
print(df)

path = 'Input_CSVs/Exp_Data/d='+str(exp_d)
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

15
[   5.05474681   74.53890398   12.24428821   16.62324988   44.33836208
  591.55992898   -4.72307241   26.04219455  -46.1040296   -52.96709878
 -146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]
       X1exp     X2exp        Yexp
0  -0.217562  0.928296    5.054747
1  -1.150787 -0.444384   74.538904
2  -1.429019  0.271155   12.244288
3  -0.965098  1.770441   16.623250
4   0.743353  0.640688   44.338362
5   0.878764  1.733711  591.559929
6  -0.501407  0.042345   -4.723072
7   0.004174  1.053848   26.042195
8   0.421099 -0.176333  -46.104030
9  -0.324795  1.314813  -52.967099
10 -0.558000  1.442000 -146.699478
11 -0.050000  0.467000  -80.767738
12  0.623000  0.028000 -108.166639
13 -0.820000  0.620000  -40.670725
14  0.220000  0.300000  -72.225084


In [8]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4", "b1", "b2", "b3", "b4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["theta1", "theta2"])


#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 0).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat)
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[i]    
all_p_emulator = df
# print(all_p_emulator)
print(all_p_standard)


          a1        a2        a3        a4        b1        b2         b3  \
0  -1.174050 -1.657363 -9.801574 -1.555507  0.656785  0.016924  14.569803   
1   0.794682  1.912550 -5.742472 -1.213031  1.198875 -1.966124   8.008349   
2  -0.663121 -0.341067 -4.649591  1.451850 -0.455949  1.643482  13.506130   
3   0.823427  0.417438 -7.354103  0.889101  1.615352  0.582054  10.314934   
4  -0.095546  0.841428 -3.097543 -0.212451 -1.825304 -0.992556   8.933613   
5  -0.475411  1.075316 -2.391048  1.984270  1.213753 -1.752558  11.339518   
6   1.880809 -0.718938 -9.133734  1.166121  0.446707 -0.734535   9.599875   
7  -1.344642 -1.214921 -4.003669  0.764113 -0.156409 -0.108014  10.691647   
8  -0.327177 -0.064581 -7.860302 -0.802933 -0.675444 -1.010933  12.437089   
9   0.479491 -0.558186 -6.802860 -1.714378 -0.958993  0.675370  12.744843   
10 -1.513783  1.531595 -8.333879 -0.481373 -0.330903 -0.296955   5.125783   
11  1.592111  1.243569 -5.448344 -0.066206  0.853414 -0.569059  11.750596   

In [10]:
print(all_p_standard.shape)
y_data = create_y_data(all_p_emulator, Constants, Xexp, skip_param_types)
theta_b = np.array([-1.03533496, -1.74325843, -0.47707726,  1.95306714, -1.05726904, -0.07808274,
 14.92232653, 1.8301121])
Xexp2 = np.array([[-0.2175623, 0.9282957]]).reshape((1,-1))
print(theta_b.shape, Xexp2.shape)
# y_data2 = create_y_data(all_p_emulator2, Constants, Xexp2, skip_param_types = 1)

y_data2 = create_y_data(theta_b, Constants, Xexp2.reshape((1,-1)), skip_param_types, noise_std)
# all_p_standard = np.array([-1,-1,-6.5,0.7,
#                            0,0,11,0.6])
sse_data = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj", skip_param_types = skip_param_types)
print(y_data[0:5])
print(y_data2)
# print(sse_data)

(20, 8)
(8,) (1, 2)
[-29.51994019  -1.0829347  -29.51994019  -1.0829347  -29.51994019]
[7.36830441e+219]


In [8]:
## Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")

#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    all_2_in_y = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj", skip_param_types=skip_param_types)
    all_2_data['Yexp'] = all_2_in_y
    
    all_2_data_ln_obj = all_p_standard.copy()
    all_2_in_y_ln_obj = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "LN_obj", skip_param_types=skip_param_types)
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_p_emulator, Constants, Xexp, skip_param_types=skip_param_types)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/d=' + str(d) +"/"+ titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'t='+len_data)+'.csv')